# Agent
 

## Agent 与 ReAct 框架

Agent 是 ReAct 框架的一种实现，理解 Agent，首先要理解 ReAct 框架。


ReAct 指 Reasoning + Acting，是模拟人类对“行动”和“推理”的一种协同，人类就是利用这种协同，去学习新任务并做出决策或推理。


已经有论文研究发现，大语言模型可以通过生成推理痕迹和任务特定行动来实现更大的协同作用。这是 ReAct 模式可行的依据。



所谓 ReAct，可以理解为，LLM 视为人类大脑，对于输入进来的任务，先让 LLM 通过充分思考进行推理，做出工作分解，形成工作流程（线性或非线性均可，依据 LLM 推理能力和实际情况），然后依次通过推理或调用工具完成任务。


> 比如说，你在运营花店的过程中，经常会经历天气变化而导致的鲜花售价变化，那么，每天早上你会如何为你的鲜花定价？
> 
> 也许你会告诉我，我会去Google上面查一查今天的鲜花成本价啊（行动 ），也就是我预计的进货的价格，然后我会根据这个价格的高低（观察 ），来确定我要加价多少（思考 ），最后计算出一个售价（行动）！
>
> LLM 通过 Agent，也可以按这个思路完成任务。


#  ZERO_SHOT_REACT

In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens=512
)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

res = agent.invoke("目前市场上玫瑰花的平均价格是多少？如果我在此基础上加价15%卖出，应该如何定价？")
print(res)

/Users/gevin/projects/pycharm/ai/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


{'input': '目前市场上玫瑰花的平均价格是多少？如果我在此基础上加价15%卖出，应该如何定价？', 'output': '每支玫瑰花的售价应该定为5.75元。'}


In [1]:
import os
from langchain_openai import AzureChatOpenAI
from langchain.agents import load_tools, create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate

LANGCHAIN_TRACING_V2 = True
LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"
LANGCHAIN_API_KEY = os.environ.get("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = "default"

template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}"""
prompt = PromptTemplate.from_template(template)

llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens=512
)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)
res = agent_executor.invoke({"input": "目前市场上玫瑰花的平均价格是多少？如果我在此基础上加价15%卖出，应该如何定价？"})
print(res)

{'input': '目前市场上玫瑰花的平均价格是多少？如果我在此基础上加价15%卖出，应该如何定价？', 'output': '玫瑰花的定价范围应该在每枝0.575元到0.69元。'}


In [2]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import load_tools, create_react_agent, AgentExecutor
from langchain import hub

prompt = hub.pull("hwchase17/react")

llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens=512
)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)
res = agent_executor.invoke({"input": "目前市场上玫瑰花的平均价格是多少？如果我在此基础上加价15%卖出，应该如何定价？"})
print(res)

{'input': '目前市场上玫瑰花的平均价格是多少？如果我在此基础上加价15%卖出，应该如何定价？', 'output': '目前市场上玫瑰花的平均价格约为21.58元，加价15%后，应该定价为24.82元。'}
